In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost
from xgboost import XGBRegressor

In [2]:
region_dict = {
    "1": "BSk",
    "9": "Dfb",
    "4": "Cfa",
    "7": "Csb",
    "8": "Dfa",
    "3": "BWk",
    "10": "Dfc",
    "2": "BWh",
    "6": "Csa",
    "11": "Dsb",
    "0": "BSh",
    "5": "Cfb",
    "12": "Dsc",
    "13": "Dwa",
    "14": "Dwb"
}

In [3]:
features = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3', 
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5',
 'diff_pres_1_first',
 'diff_pres_1_min',
 'diff_pres_1_max',
 'range_pres_1',
 'scale_pres_1',
 'diff_ccsm3_month_1_first',
 'diff_ccsm3_month_1_min',
 'diff_ccsm3_month_1_max',
 'range_ccsm3_month_1',
 'scale_ccsm3_month_1',
 'diff_sst_1_month_1_first',
 'diff_sst_1_month_1_min',
 'diff_sst_1_month_1_max',
 'range_sst_1_month_1',
 'scale_sst_1_month_1',
 'sst_1_lag_1',
 'sst_5_lag_1',
 'sst_4_lag_1',
 'sst_1_lag_1_season',
 'sst_5_lag_1_season',
 'sst_4_lag_1_season']

target=["contest-tmp2m-14d__tmp2m"]

slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_500 = 'contest-wind-h500-14d__wind-hgt-500'
sst_1_col = 'sst-2010-1'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'
group_cols = ['startdate']

In [4]:
train_df = pd.read_csv('./train_h2o.csv')
test_df = pd.read_csv('./test_h2o.csv')

In [5]:
train_df['index'] = np.nan

In [6]:
sst_features_5 = []
for i in [1, 5, 3, 4]:
    sst_features_5 = sst_features_5 + [f'sst_{i}_neigh_5_mean', f'sst_{i}_neigh_5_min', f'sst_{i}_neigh_5_max']

sst_features_5_diag_pos = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_pos = sst_features_5_diag_pos + [f'sst_{i}_neigh_5_diag_pos_mean', f'sst_{i}_neigh_5_diag_pos_min', f'sst_{i}_neigh_5_diag_pos_max']


sst_features_time_5 = []
for i in [1, 5, 3, 4]:
    sst_features_time_5 = sst_features_time_5 + [f'sst_{i}_time_5_mean', f'sst_{i}_time_5_min', f'sst_{i}_time_5_max']

sst_features_time_7 = []
for i in [1, 5, 3, 4]:
    sst_features_time_7 = sst_features_time_7 + [f'sst_{i}_time_7_mean', f'sst_{i}_time_7_min', f'sst_{i}_time_7_max']


sst_features_time_14 = []
for i in [1, 5, 3, 4]:
    sst_features_time_14 = sst_features_time_14 + [f'sst_{i}_time_14_mean', f'sst_{i}_time_14_min', f'sst_{i}_time_14_max']

sst_features_time_9 = []
for i in [1, 5, 3, 4]:
    sst_features_time_9 = sst_features_time_9 + [f'sst_{i}_time_9_mean', f'sst_{i}_time_9_min', f'sst_{i}_time_9_max']


sst_features_time_11 = []
for i in [1, 5, 3, 4]:
    sst_features_time_11 = sst_features_time_11 + [f'sst_{i}_time_11_mean', f'sst_{i}_time_11_min', f'sst_{i}_time_11_max']

sst_features_11 = []
for i in [1, 5, 3, 4]:
    sst_features_11 = sst_features_11 + [f'sst_{i}_neigh_11_mean', f'sst_{i}_neigh_11_min', f'sst_{i}_neigh_11_max']

sst_features_3 = []
for i in [1, 5, 3, 4]:
    sst_features_3 = sst_features_3 + [f'sst_{i}_neigh_3_mean', f'sst_{i}_neigh_3_min', f'sst_{i}_neigh_3_max']

sst_features_7 = []
for i in [1, 5, 3, 4]:
    sst_features_7 = sst_features_7 + [f'sst_{i}_neigh_7_mean', f'sst_{i}_neigh_7_min', f'sst_{i}_neigh_7_max']

sst_features_5_lat = []
for i in [1, 5, 3, 4]:
    sst_features_5_lat = sst_features_5_lat + [f'sst_{i}_neigh_5_lat_mean', f'sst_{i}_neigh_5_lat_min', f'sst_{i}_neigh_5_lat_max']

sst_features_5_lon = []
for i in [1, 5, 3, 4]:
    sst_features_5_lon = sst_features_5_lon + [f'sst_{i}_neigh_5_lon_mean', f'sst_{i}_neigh_5_lon_min', f'sst_{i}_neigh_5_lon_max']

sst_features_9_lon = []
for i in [1, 5, 3, 4]:
    sst_features_9_lon = sst_features_9_lon + [f'sst_{i}_neigh_9_lon_mean', f'sst_{i}_neigh_9_lon_min', f'sst_{i}_neigh_9_lon_max']

sst_features_11_lon = []
for i in [1, 5, 3, 4]:
    sst_features_11_lon = sst_features_11_lon + [f'sst_{i}_neigh_11_lon_mean', f'sst_{i}_neigh_11_lon_min', f'sst_{i}_neigh_11_lon_max']

sst_features_15_lon = []
for i in [1, 5, 3, 4]:
    sst_features_15_lon = sst_features_15_lon + [f'sst_{i}_neigh_15_lon_mean', f'sst_{i}_neigh_15_lon_min', f'sst_{i}_neigh_15_lon_max']

sst_features_11_lat = []
for i in [1, 5, 3, 4]:
    sst_features_11_lat = sst_features_11_lat + [f'sst_{i}_neigh_11_lat_mean', f'sst_{i}_neigh_11_lat_min', f'sst_{i}_neigh_11_lat_max']

sst_features_9_lat = []
for i in [1, 5, 3, 4]:
    sst_features_9_lat = sst_features_9_lat + [f'sst_{i}_neigh_9_lat_mean', f'sst_{i}_neigh_9_lat_min', f'sst_{i}_neigh_9_lat_max']
    
sst_features_9 = []
for i in [1, 5, 3, 4]:
    sst_features_9 = sst_features_9 + [f'sst_{i}_neigh_9_mean', f'sst_{i}_neigh_9_min', f'sst_{i}_neigh_9_max']

sst_features_13 = []
for i in [1, 5, 3, 4]:
    sst_features_13 = sst_features_13 + [f'sst_{i}_neigh_13_mean', f'sst_{i}_neigh_13_min', f'sst_{i}_neigh_13_max']


poten_feature_5 = ['poten_neigh_5_mean', 'poten_neigh_5_min', 'poten_neigh_5_max']
poten_feature_time_5 = ['poten_time_5_mean', 'poten_time_5_min', 'poten_time_5_max']
poten_feature_time_7 = ['poten_time_7_mean', 'poten_time_7_min', 'poten_time_7_max']
poten_feature_time_11 = ['poten_time_11_mean', 'poten_time_11_min', 'poten_time_11_max']
poten_feature_time_9 = ['poten_time_9_mean', 'poten_time_9_min', 'poten_time_9_max']

poten_feature_3 = ['poten_neigh_3_mean', 'poten_neigh_3_min', 'poten_neigh_3_max']
poten_feature_7 = ['poten_neigh_7_mean', 'poten_neigh_7_min', 'poten_neigh_7_max']
poten_feature_9 = ['poten_neigh_9_mean', 'poten_neigh_9_min', 'poten_neigh_9_max']
poten_feature_11 = ['poten_neigh_11_mean', 'poten_neigh_11_min', 'poten_neigh_11_max']
poten_feature_13 = ['poten_neigh_13_mean', 'poten_neigh_13_min', 'poten_neigh_13_max']



rhum_feature_5 = ['rhum_neigh_5_mean', 'rhum_neigh_5_min', 'rhum_neigh_5_max']
rhum_feature_5_diag_pos = ['rhum_neigh_5_diag_pos_mean', 'rhum_neigh_5_diag_pos_min', 'rhum_neigh_5_diag_pos_max']

rhum_feature_5_lat = ['rhum_neigh_5_lat_mean', 'rhum_neigh_5_lat_min', 'rhum_neigh_5_lat_max']
rhum_feature_5_lon = ['rhum_neigh_5_lon_mean', 'rhum_neigh_5_lon_min', 'rhum_neigh_5_lon_max']
rhum_feature_9_lon = ['rhum_neigh_9_lon_mean', 'rhum_neigh_9_lon_min', 'rhum_neigh_9_lon_max']
rhum_feature_11_lon = ['rhum_neigh_11_lon_mean', 'rhum_neigh_11_lon_min', 'rhum_neigh_11_lon_max']
rhum_feature_15_lon = ['rhum_neigh_15_lon_mean', 'rhum_neigh_15_lon_min', 'rhum_neigh_15_lon_max']
rhum_feature_11_lat = ['rhum_neigh_11_lat_mean', 'rhum_neigh_11_lat_min', 'rhum_neigh_11_lat_max']
rhum_feature_9_lat = ['rhum_neigh_9_lat_mean', 'rhum_neigh_9_lat_min', 'rhum_neigh_9_lat_max']
rhum_feature_time_3 = ['rhum_time_3_mean', 'rhum_time_3_min', 'rhum_time_3_max']
rhum_feature_time_5 = ['rhum_time_5_mean', 'rhum_time_5_min', 'rhum_time_5_max']
rhum_feature_time_7 = ['rhum_time_7_mean', 'rhum_time_7_min', 'rhum_time_7_max']
rhum_feature_time_11 = ['rhum_time_11_mean', 'rhum_time_11_min', 'rhum_time_11_max']
rhum_feature_time_7_mid = ['rhum_time_7_mid_mean', 'rhum_time_7_mid_min', 'rhum_time_7_mid_max']
rhum_feature_time_14 = ['rhum_time_14_mean', 'rhum_time_14_min', 'rhum_time_14_max']
rhum_feature_time_21 = ['rhum_time_21_mean', 'rhum_time_21_min', 'rhum_time_21_max']
rhum_feature_time_28 = ['rhum_time_28_mean', 'rhum_time_28_min', 'rhum_time_28_max']
rhum_feature_time_9 = ['rhum_time_9_mean', 'rhum_time_9_min', 'rhum_time_9_max']

rhum_feature_3 = ['rhum_neigh_3_mean', 'rhum_neigh_3_min', 'rhum_neigh_3_max']
rhum_feature_7 = ['rhum_neigh_7_mean', 'rhum_neigh_7_min', 'rhum_neigh_7_max']
rhum_feature_9 = ['rhum_neigh_9_mean', 'rhum_neigh_9_min', 'rhum_neigh_9_max']
rhum_feature_11 = ['rhum_neigh_11_mean', 'rhum_neigh_11_min', 'rhum_neigh_11_max']
rhum_feature_13 = ['rhum_neigh_13_mean', 'rhum_neigh_13_min', 'rhum_neigh_13_max']


slp_feature_5 = ['slp_neigh_5_mean', 'slp_neigh_5_min', 'slp_neigh_5_max']
slp_feature_5_diag_pos = ['slp_neigh_5_diag_pos_mean', 'slp_neigh_5_diag_pos_min', 'slp_neigh_5_diag_pos_max']
slp_feature_5_lat = ['slp_neigh_5_lat_mean', 'slp_neigh_5_lat_min', 'slp_neigh_5_lat_max']
slp_feature_5_lon = ['slp_neigh_5_lon_mean', 'slp_neigh_5_lon_min', 'slp_neigh_5_lon_max']
slp_feature_9_lon = ['slp_neigh_9_lon_mean', 'slp_neigh_9_lon_min', 'slp_neigh_9_lon_max']
slp_feature_11_lon = ['slp_neigh_11_lon_mean', 'slp_neigh_11_lon_min', 'slp_neigh_11_lon_max']
slp_feature_15_lon = ['slp_neigh_15_lon_mean', 'slp_neigh_15_lon_min', 'slp_neigh_15_lon_max']
slp_feature_11_lat = ['slp_neigh_11_lat_mean', 'slp_neigh_11_lat_min', 'slp_neigh_11_lat_max']
slp_feature_9_lat = ['slp_neigh_9_lat_mean', 'slp_neigh_9_lat_min', 'slp_neigh_9_lat_max']
slp_feature_time_5 = ['slp_time_5_mean', 'slp_time_5_min', 'slp_time_5_max']
slp_feature_time_7 = ['slp_time_7_mean', 'slp_time_7_min', 'slp_time_7_max']
slp_feature_time_11 = ['slp_time_11_mean', 'slp_time_11_min', 'slp_time_11_max']
slp_feature_time_14 = ['slp_time_14_mean', 'slp_time_14_min', 'slp_time_14_max']
slp_feature_time_9 = ['slp_time_9_mean', 'slp_time_9_min', 'slp_time_9_max']

slp_feature_3 = ['slp_neigh_3_mean', 'slp_neigh_3_min', 'slp_neigh_3_max']
slp_feature_7 = ['slp_neigh_7_mean', 'slp_neigh_7_min', 'slp_neigh_7_max']
slp_feature_9 = ['slp_neigh_9_mean', 'slp_neigh_9_min', 'slp_neigh_9_max']
slp_feature_11 = ['slp_neigh_11_mean', 'slp_neigh_11_min', 'slp_neigh_11_max']
slp_feature_13 = ['slp_neigh_13_mean', 'slp_neigh_13_min', 'slp_neigh_13_max']


precip_feature_5 = ['precip_neigh_5_mean', 'precip_neigh_5_min', 'precip_neigh_5_max']
precip_feature_5_diag_pos = ['precip_neigh_5_diag_pos_mean', 'precip_neigh_5_diag_pos_min', 'precip_neigh_5_diag_pos_max']
precip_feature_5_lat = ['precip_neigh_5_lat_mean', 'precip_neigh_5_lat_min', 'precip_neigh_5_lat_max']
precip_feature_5_lon = ['precip_neigh_5_lon_mean', 'precip_neigh_5_lon_min', 'precip_neigh_5_lon_max']
precip_feature_9_lon = ['precip_neigh_9_lon_mean', 'precip_neigh_9_lon_min', 'precip_neigh_9_lon_max']
precip_feature_11_lon = ['precip_neigh_11_lon_mean', 'precip_neigh_11_lon_min', 'precip_neigh_11_lon_max']
precip_feature_15_lon = ['precip_neigh_15_lon_mean', 'precip_neigh_15_lon_min', 'precip_neigh_15_lon_max']
precip_feature_11_lat = ['precip_neigh_11_lat_mean', 'precip_neigh_11_lat_min', 'precip_neigh_11_lat_max']
precip_feature_9_lat = ['precip_neigh_9_lat_mean', 'precip_neigh_9_lat_min', 'precip_neigh_9_lat_max']
precip_feature_time_3 = ['precip_time_3_mean', 'precip_time_3_min', 'precip_time_3_max']
precip_feature_time_5 = ['precip_time_5_mean', 'precip_time_5_min', 'precip_time_5_max']
precip_feature_time_7 = ['precip_time_7_mean', 'precip_time_7_min', 'precip_time_7_max']
precip_feature_time_11 = ['precip_time_11_mean', 'precip_time_11_min', 'precip_time_11_max']
precip_feature_time_7_mid = ['precip_time_7_mid_mean', 'precip_time_7_mid_min', 'precip_time_7_mid_max']
precip_feature_time_9 = ['precip_time_9_mean', 'precip_time_9_min', 'precip_time_9_max']
precip_feature_time_14 = ['precip_time_14_mean', 'precip_time_14_min', 'precip_time_14_max']
precip_feature_time_21 = ['precip_time_21_mean', 'precip_time_21_min', 'precip_time_21_max']
precip_feature_time_28 = ['precip_time_28_mean', 'precip_time_28_min', 'precip_time_28_max']

precip_feature_3 = ['precip_neigh_3_mean', 'precip_neigh_3_min', 'precip_neigh_3_max']
precip_feature_7 = ['precip_neigh_7_mean', 'precip_neigh_7_min', 'precip_neigh_7_max']
precip_feature_9 = ['precip_neigh_9_mean', 'precip_neigh_9_min', 'precip_neigh_9_max']
precip_feature_11 = ['precip_neigh_11_mean', 'precip_neigh_11_min', 'precip_neigh_11_max']
precip_feature_13 = ['precip_neigh_13_mean', 'precip_neigh_13_min', 'precip_neigh_13_max']


pres_feature_5 = ['pres_neigh_5_mean', 'pres_neigh_5_min', 'pres_neigh_5_max']
pres_feature_5_diag_pos = ['pres_neigh_5_diag_pos_mean', 'pres_neigh_5_diag_pos_min', 'pres_neigh_5_diag_pos_max']
pres_feature_5_lat = ['pres_neigh_5_lat_mean', 'pres_neigh_5_lat_min', 'pres_neigh_5_lat_max']
pres_feature_5_lon = ['pres_neigh_5_lon_mean', 'pres_neigh_5_lon_min', 'pres_neigh_5_lon_max']
pres_feature_9_lon = ['pres_neigh_9_lon_mean', 'pres_neigh_9_lon_min', 'pres_neigh_9_lon_max']
pres_feature_15_lon = ['pres_neigh_15_lon_mean', 'pres_neigh_15_lon_min', 'pres_neigh_15_lon_max']
pres_feature_11_lat = ['pres_neigh_11_lat_mean', 'pres_neigh_11_lat_min', 'pres_neigh_11_lat_max']
pres_feature_9_lat = ['pres_neigh_9_lat_mean', 'pres_neigh_9_lat_min', 'pres_neigh_9_lat_max']
pres_feature_time_3 = ['pres_time_3_mean', 'pres_time_3_min', 'pres_time_3_max']
pres_feature_time_5 = ['pres_time_5_mean', 'pres_time_5_min', 'pres_time_5_max']
pres_feature_time_7 = ['pres_time_7_mean', 'pres_time_7_min', 'pres_time_7_max']
pres_feature_time_11 = ['pres_time_11_mean', 'pres_time_11_min', 'pres_time_11_max']
pres_feature_time_7_mid = ['pres_time_7_mid_mean', 'pres_time_7_mid_min', 'pres_time_7_mid_max']
pres_feature_time_9 = ['pres_time_9_mean', 'pres_time_9_min', 'pres_time_9_max']
pres_feature_time_14 = ['pres_time_14_mean', 'pres_time_14_min', 'pres_time_14_max']
pres_feature_time_21 = ['pres_time_21_mean', 'pres_time_21_min', 'pres_time_21_max']
pres_feature_time_28 = ['pres_time_28_mean', 'pres_time_28_min', 'pres_time_28_max']

pres_feature_3 = ['pres_neigh_3_mean', 'pres_neigh_3_min', 'pres_neigh_3_max']
pres_feature_7 = ['pres_neigh_7_mean', 'pres_neigh_7_min', 'pres_neigh_7_max']
pres_feature_9 = ['pres_neigh_9_mean', 'pres_neigh_9_min', 'pres_neigh_9_max']
pres_feature_11 = ['pres_neigh_11_mean', 'pres_neigh_11_min', 'pres_neigh_11_max']
pres_feature_13 = ['pres_neigh_13_mean', 'pres_neigh_13_min', 'pres_neigh_13_max']



wind_feature_11 = ['wind_500_neigh_11_mean', 'wind_500_neigh_11_min', 'wind_500_neigh_11_max']
wind_feature_9 = ['wind_500_neigh_9_mean', 'wind_500_neigh_9_min', 'wind_500_neigh_9_max']
wind_feature_5 = ['wind_500_neigh_5_mean', 'wind_500_neigh_5_min', 'wind_500_neigh_5_max']
wind_feature_5_diag_pos = ['wind_500_neigh_5_diag_pos_mean', 'wind_500_neigh_5_diag_pos_min', 'wind_500_neigh_5_diag_pos_max']
wind_feature_5_lat = ['wind_500_neigh_5_lat_mean', 'wind_500_neigh_5_lat_min', 'wind_500_neigh_5_lat_max']
wind_feature_5_lon = ['wind_500_neigh_5_lon_mean', 'wind_500_neigh_5_lon_min', 'wind_500_neigh_5_lon_max']
wind_feature_9_lon = ['wind_500_neigh_9_lon_mean', 'wind_500_neigh_9_lon_min', 'wind_500_neigh_9_lon_max']
wind_feature_11_lon = ['wind_500_neigh_11_lon_mean', 'wind_500_neigh_11_lon_min', 'wind_500_neigh_11_lon_max']
wind_feature_15_lon = ['wind_500_neigh_15_lon_mean', 'wind_500_neigh_15_lon_min', 'wind_500_neigh_15_lon_max']
wind_feature_11_lat = ['wind_500_neigh_11_lat_mean', 'wind_500_neigh_11_lat_min', 'wind_500_neigh_11_lat_max']
wind_feature_9_lat = ['wind_500_neigh_9_lat_mean', 'wind_500_neigh_9_lat_min', 'wind_500_neigh_9_lat_max']
wind_feature_time_5 = ['wind_500_time_5_mean', 'wind_500_time_5_min', 'wind_500_time_5_max']
wind_feature_time_7 = ['wind_500_time_7_mean', 'wind_500_time_7_min', 'wind_500_time_7_max']
wind_feature_time_14 = ['wind_500_time_14_mean', 'wind_500_time_14_min', 'wind_500_time_14_max']
wind_feature_time_9 = ['wind_500_time_9_mean', 'wind_500_time_9_min', 'wind_500_time_9_max']
wind_feature_time_11 = ['wind_500_time_11_mean', 'wind_500_time_11_min', 'wind_500_time_11_max']
wind_feature_7 = ['wind_500_neigh_7_mean', 'wind_500_neigh_7_min', 'wind_500_neigh_7_max']
wind_feature_3 = ['wind_500_neigh_3_mean', 'wind_500_neigh_3_min', 'wind_500_neigh_3_max']
wind_feature_13 = ['wind_500_neigh_13_mean', 'wind_500_neigh_13_min', 'wind_500_neigh_13_max']

In [7]:
selected_cols = features + sst_features_5 + wind_feature_5 + slp_feature_5 + precip_feature_5 + pres_feature_5 + rhum_feature_5 + slp_feature_3 + precip_feature_3 + pres_feature_3  + slp_feature_7 + rhum_feature_7 + sst_features_7 + precip_feature_7 + pres_feature_7 + wind_feature_7 + slp_feature_9 + wind_feature_9 + pres_feature_9 + precip_feature_9 + rhum_feature_9 + rhum_feature_13 + \
rhum_feature_time_7 + pres_feature_time_7 + precip_feature_time_7 + \
rhum_feature_time_14 + pres_feature_time_14 + precip_feature_time_14 + \
sst_features_5_lat + wind_feature_5_lat + slp_feature_5_lat + precip_feature_5_lat + pres_feature_5_lat + rhum_feature_5_lat + \
sst_features_5_lon + wind_feature_5_lon + slp_feature_5_lon + precip_feature_5_lon + pres_feature_5_lon + rhum_feature_5_lon + \
slp_feature_5_diag_pos + precip_feature_5_diag_pos + pres_feature_5_diag_pos + rhum_feature_5_diag_pos

In [8]:
cols = selected_cols

In [9]:
def train_test_by_df(train_df_, test_df_, iterations):
  X = train_df_[cols].values
  y = train_df_[target[0]].values

  skf = KFold(n_splits=5, random_state=2048, shuffle=True)
  skf.get_n_splits(X, y)

  train_index_list = []
  test_index_list = []
  train_x_list = []
  val_x_list = []
  train_y_list = []
  val_y_list = []

  for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index]) 
  
    
  cat_models = []
  for i in tqdm(range(5)):
    train_x, val_x = train_x_list[i], val_x_list[i]
    train_y, val_y = train_y_list[i], val_y_list[i]
    clf = CatBoostRegressor(iterations=iterations, verbose=1000, task_type="GPU", 
                        devices='0')
    clf.fit(train_x, train_y, eval_set=[(val_x, val_y)])
    cat_models.append(clf)

  preds = []
  for cat_model in tqdm(cat_models):
      preds.append(cat_model.predict(test_df_[cols].values))
  s = 0
  for pred in preds:
      s += pred
  s = s/5
  
  return s, cat_models


In [ ]:
s, cat_models = train_test_by_df(train_df, test_df, 290000)

5it [01:01, 12.28s/it]
  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.022853
0:	learn: 9.6701521	test: 9.7028248	best: 9.7028248 (0)	total: 10.3ms	remaining: 49m 49s
1000:	learn: 0.8735539	test: 0.8830492	best: 0.8830492 (1000)	total: 6.08s	remaining: 29m 16s
2000:	learn: 0.6457986	test: 0.6594665	best: 0.6594665 (2000)	total: 12.1s	remaining: 28m 55s
3000:	learn: 0.5428084	test: 0.5588157	best: 0.5588157 (3000)	total: 18.1s	remaining: 28m 47s
4000:	learn: 0.4797057	test: 0.4974733	best: 0.4974733 (4000)	total: 24.2s	remaining: 28m 46s
5000:	learn: 0.4353871	test: 0.4548150	best: 0.4548150 (5000)	total: 30.1s	remaining: 28m 37s
6000:	learn: 0.4018547	test: 0.4226729	best: 0.4226729 (6000)	total: 36s	remaining: 28m 23s


In [ ]:
output_df = pd.DataFrame(columns = ['index', 'pred'])
output_df['index'] = [int(i) for i in test_df['index']]
output_df['pred'] = s
res_df = pd.read_csv('./sample_solution.csv')
res_df = res_df.merge(output_df, how='left', on = ['index'])
res_df['contest-tmp2m-14d__tmp2m'] = res_df['pred']
res_df[[target[0], 'index']].to_csv('submission_fe_pos_v3.csv', index=False)

In [ ]:
res_df = pd.read_csv('./sample_solution.csv')
cat_df = pd.read_csv('./submission_fe_pos_v2.csv')
lgb_df = pd.read_csv('./best_lgb.csv')
xgb_df = pd.read_csv('./best_xgb.csv')
s = cat_df[target[0]] * 0.7 + lgb_df[target[0]] * 0.25 + xgb_df[target[0]] * 0.05
res_df[target[0]] = s
res_df[[target[0], 'index']].to_csv('submission_ensemble.csv', index=False)